# Adjusting LC Plans

## 1. go to parsing_cenko.ipynb for how I found the abs mag peak range, with redshift, beta, k-correction

:) 

## 2. Spectral Index Across Filters

In [ ]:
# -------------------------------------------
# This could go in the shared_utils py file. 
# -------------------------------------------

def apply_spectral_index(mag_ref, filtername, ref_filter=None, beta=None):
    """
    Adjust magnitude based on spectral index β (Fν ∝ ν^β) to translate
    from reference filter to target filter.

    Parameters
    ----------
    mag_ref : float or array_like
        Magnitude in the reference filter (e.g., r-band).
    filtername : str
        Target LSST filter (e.g., 'g', 'i').
    ref_filter : str or None
        Reference filter used to simulate the light curve. Default is from config.
    beta : float or None
        Spectral index. If None, uses value from GRB_CONFIG.

    Returns
    -------
    mag_target : float or array_like
        Adjusted magnitude in the target filter.
    """
    FILTER_CENTRAL_FREQS = {
    'u': 8.088e+14,
    'g': 6.293e+14,
    'r': 4.844e+14,
    'i': 3.979e+14,
    'z': 3.461e+14,
    'y': 3.080e+14,
    }
    
    if ref_filter is None:
        ref_filter = GRB_CONFIG["reference_filter"]
    if beta is None:
        beta = GRB_CONFIG["spectral_index_beta"]

    if filtername == ref_filter:
        return mag_ref

    nu_ref = FILTER_CENTRAL_FREQS[ref_filter]
    nu_target = FILTER_CENTRAL_FREQS[filtername]
    delta_mag = 2.5 * beta * np.log10(nu_target / nu_ref)
    return mag_ref + delta_mag

# -------------------------------------------
# This would go in LC.interp in the metric file
# -------------------------------------------

from shared_utils import apply_spectral_index

def interp(self, t, filtername, lc_indx=0):
    mag_r = np.interp(
        t,
        self.data[lc_indx]["r"]['ph'],
        self.data[lc_indx]["r"]['mag'],
        left=99, right=99
    )
    return apply_spectral_index(mag_r, filtername)
    

# -------------------------------------------
# Global configuration for GRB afterglow modeling in script
# -------------------------------------------

GRB_CONFIG = {
    "spectral_index_beta": -0.75,
    "reference_filter": "r"
}




In [52]:
import os
import numpy as np
import pandas as pd
from rubin_scheduler.data import get_data_dir
from rubin_sim.phot_utils.bandpass import Bandpass

# LSST filters
filters = ['u', 'g', 'r', 'i', 'z', 'y']

# Load throughput curves and compute effective λ and ν
data_dir = get_data_dir()
throughput_dir = os.path.join(data_dir, 'throughputs', 'baseline')
results = []

for f in filters:
    filepath = os.path.join(throughput_dir, f'total_{f}.dat')
    bp = Bandpass()
    bp.read_throughput(filepath)

    # Effective wavelength
    lam_eff = float(bp.calc_eff_wavelen()[0])  # unpack properly
    nu_c = 3e8 / (lam_eff * 1e-9)  # Hz


    results.append((f, lam_eff, nu_c))

# Make table
df = pd.DataFrame(results, columns=["Filter", "Eff_Wavelength_nm", "Central_Frequency_Hz"])
df["Eff_Wavelength_nm"] = df["Eff_Wavelength_nm"].map("{:.1f}".format)
df["Central_Frequency_Hz"] = df["Central_Frequency_Hz"].map("{:.3e}".format)

print(df.to_markdown(index=False))

# Create dict with formatted central frequencies (3 sig figs, scientific notation)
print("FILTER_CENTRAL_FREQS = {")
for f, _, nu in results:
    print(f"    '{f}': {nu:.3e},")
print("}")



| Filter   |   Eff_Wavelength_nm |   Central_Frequency_Hz |
|:---------|--------------------:|-----------------------:|
| u        |               370.9 |              8.088e+14 |
| g        |               476.7 |              6.293e+14 |
| r        |               619.4 |              4.844e+14 |
| i        |               753.9 |              3.979e+14 |
| z        |               866.8 |              3.461e+14 |
| y        |               973.9 |              3.08e+14  |
FILTER_CENTRAL_FREQS = {
    'u': 8.088e+14,
    'g': 6.293e+14,
    'r': 4.844e+14,
    'i': 3.979e+14,
    'z': 3.461e+14,
    'y': 3.080e+14,
}


## 3. Modeling On-Axis and Off-Axis 

In [ ]:
# Add to configuration in script ipynb

GRB_CONFIG = {
    "population_type": "mixed",  # options: "on_axis", "off_axis", "mixed"
    "on_axis_rate": 1e-9,
    "off_axis_rate": 1e-8,
    "theta_jet_deg": 5.0,
    "theta_obs_range_deg": [5.1, 45],  # for off-axis only
    ...
}


**We would need to do the following elsewhere to handle a population type toggle:**

Update generateGRBPopSlicer() to:

Apply the population_type

Generate two separate subpopulations if "mixed" is selected

Assign a field like is_off_axis and theta_obs to each event accordingly and save to obs_data

In [60]:
def granot_transform(t_on, mag_on, theta_obs_deg, theta_jet_deg=5.0):
    """
    Transform an on-axis GRB afterglow light curve to off-axis geometry using 
    the analytic approximation from Granot et al. (2002).
    
    The observed flux for an off-axis observer is both:
    - dimmer: due to relativistic beaming effects
    - delayed: because photons arrive from wider angles with longer path lengths

    We approximate these effects by shifting the light curve as:

        t_off = t_on * b
        mag_off = mag_on + 2.5 * log10(a)

    where:
        - a = (delta_theta / theta_jet)^3 + 1  ≈ dimming factor
        - b = 1 + 5 * (delta_theta / theta_jet)^2 ≈ time delay factor
        - delta_theta = theta_obs - theta_jet

    This captures the qualitative behavior shown in Figure 1 of Granot 2002,
    where even modest offsets in viewing angle result in significantly fainter and delayed light curves.

    Parameters
    ----------
    t_on : array_like
        Time array for the on-axis light curve (days).
    mag_on : array_like
        Magnitude array for the on-axis light curve.
    theta_obs_deg : float
        Viewing angle of observer relative to jet axis, in degrees.
    theta_jet_deg : float, optional
        Jet half-opening angle, in degrees (default is 5.0 degrees).

    Returns
    -------
    t_off : ndarray
        Time array for off-axis light curve.
    mag_off : ndarray
        Magnitude array for off-axis light curve (fainter and delayed).
    
    References
    ----------
    Granot, Panaitescu, Kumar, & Woosley (2002), 
    "Off-Axis Afterglow Emission from Jetted Gamma-Ray Bursts", 
    Astrophysical Journal Letters, 570:L61–L64. https://doi.org/10.1086/340991
    """
    theta_obs = np.radians(theta_obs_deg)
    theta_jet = np.radians(theta_jet_deg)
    delta_theta = theta_obs - theta_jet

    # Delay factor b = 1 + 5 * (delta_theta / theta_jet)^2
    b = 1 + 5 * (delta_theta / theta_jet)**2

    # Dimming factor  a = (delta_theta / theta_jet)^3 + 1 
    a = (delta_theta / theta_jet)**3 + 1

    # Apply delay and dimming to light curve
    t_off = t_on * b
    mag_off = mag_on + 2.5 * np.log10(a)

    return t_off, mag_off

Something like this in population generator: 

In [ ]:
if slice_point.get("is_off_axis", False):
    theta_obs = slice_point["theta_obs"]
    t_r, mag_r = granot_transform(
        self.data[lc_indx]["r"]["ph"],
        self.data[lc_indx]["r"]["mag"],
        theta_obs_deg=theta_obs,
        theta_jet_deg=GRB_CONFIG["theta_jet_deg"]
    )
else:
    t_r = self.data[lc_indx]["r"]["ph"]
    mag_r = self.data[lc_indx]["r"]["mag"]
